# Convolutional Networks in Tensorflow

This notebook will show you how to implement convolutional neural networks in Tensorflow using the Keras API.

In [ ]:
from glob import glob

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, utils

## Step 1. Organize the data files

We will use the [Caltech101](http://www.vision.caltech.edu/Image_Datasets/Caltech101/) data set that contains pictures of objects belonging to 101 categories. The data set has already been downloaded to the `/data400/101_ObjectCategories/` shared directory.

First, let's determine what the 101 categories are.

In [ ]:
categories = [s.split("/")[-1] for s in glob("/data400/101_ObjectCategories/*")]
print(len(categories))
categories[:10]

Now let's get a list of all of the images in all categories.

In [ ]:
images = []
for cat_id, cat in enumerate(categories):
    images.extend(
        (cat_id, img) for img in
         glob("/data400/101_ObjectCategories/%s/*.jpg" % cat)
    )
    
print(len(images))
np.random.shuffle(images)
images[:10]

## Step 2. Set up the model in Tensorflow

Now we set up the Tensorflow model. We use the Keras API. Since our model is just a sequence of layers, one after the other, we use `tf.keras.Sequential` to set up our model. This takes as input a list of layers. We have:

- convolutional layer: [`Conv2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D), since these are images 
- max pooling layers: [`MaxPool2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)
- flatten: [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), which takes an N-D tensor and turns it into a 2-D tensor, with one dimension for each image and another for everything else.
- dense layers: [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense).

We use ReLU as the activation function, except in the last layer where we use softmax. (Why?)

In [ ]:
model = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                  data_format='channels_last',
                  activation='relu',
                  input_shape=(60, 45, 3)),
    layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(categories), activation='softmax')
])

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

## Step 3. Read in the data and train the model.

Now we'll read in each image, decode it, and resize it to a 60 x 45 x 3 tensor. (Each image is 60 x 45, with 3 color channels.)

We also need to keep track of the corresponding category id (`cat_id`) of each image.

In [ ]:
# Decode images one by one and resize them to 60 x 45.
# (We also keep track of the actual cat_id of each image.)
imgs = []
cat_ids = []
for cat_id, img_file in images:
    img_decoded = tf.image.decode_jpeg(
        tf.read_file(img_file),
        channels=3
    )
    img_resized = tf.image.resize_images(img_decoded, [60, 45])
    imgs.append(img_resized)
    cat_ids.append(cat_id)

Now each image inside `imgs` is a 60 x 45 x 3 tensor. We'll stack these images along a 4th dimension to produce a 4D tensor. Now we will have different images along the `axis=0` dimension.

In [ ]:
# Each img_resized is a 3D tensor. Stack them along axis=0 to 
# produce a 4D tensor. (The new dimension is just the batch size.)
x_train = tf.stack(imgs[:4000], axis=0)

# cat_ids are integers from 0 to len(categories)
y_train = utils.to_categorical(cat_ids[:4000], len(categories))

In [ ]:
# Now fit the model.
model.fit(x_train, y_train, verbose=1, epochs=10, steps_per_epoch=100)

## Model Evaluation

We need to evaluate the model on test data.

In [ ]:
x_test = tf.stack(imgs[4000:], axis=0)

# cat_ids are integers from 0 to len(categories)
y_test = utils.to_categorical(cat_ids[4000:], len(categories))

model.evaluate(x_test, y_test)